In [20]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

# 0. Initial Data Loading and Exploration

## HDI

In [21]:
hdi_df = pd.read_csv('data/HDR21-22_Composite_indices_complete_time_series.csv')
hdi_df.sample(3)

,iso3,country,hdicode,region,hdi_rank_2021,hdi_1990,hdi_1991,hdi_1992,hdi_1993,hdi_1994,...,mf_2012,mf_2013,mf_2014,mf_2015,mf_2016,mf_2017,mf_2018,mf_2019,mf_2020,mf_2021
120,MOZ,Mozambique,Low,SSA,185.0,0.238,0.240,0.234,0.239,0.242,...,3.24,3.00,3.63,3.51,3.28,3.28,2.18,2.19,2.19,2.19
97,LBN,Lebanon,High,AS,112.0,NaN,NaN,NaN,NaN,NaN,...,19.73,18.94,16.94,15.45,14.75,14.40,13.10,12.40,12.40,12.40
28,CAF,Central African Republic,Low,SSA,188.0,0.338,0.333,0.319,0.319,0.321,...,2.40,2.08,2.11,2.07,2.12,2.22,1.49,1.57,1.57,1.57


In [22]:
# Exploring the columns
hdi_df.columns.tolist()

['iso3',
 'country',
 'hdicode',
 'region',
 'hdi_rank_2021',
 'hdi_1990',
 'hdi_1991',
 'hdi_1992',
 'hdi_1993',
 'hdi_1994',
 'hdi_1995',
 'hdi_1996',
 'hdi_1997',
 'hdi_1998',
 'hdi_1999',
 'hdi_2000',
 'hdi_2001',
 'hdi_2002',
 'hdi_2003',
 'hdi_2004',
 'hdi_2005',
 'hdi_2006',
 'hdi_2007',
 'hdi_2008',
 'hdi_2009',
 'hdi_2010',
 'hdi_2011',
 'hdi_2012',
 'hdi_2013',
 'hdi_2014',
 'hdi_2015',
 'hdi_2016',
 'hdi_2017',
 'hdi_2018',
 'hdi_2019',
 'hdi_2020',
 'hdi_2021',
 'le_1990',
 'le_1991',
 'le_1992',
 'le_1993',
 'le_1994',
 'le_1995',
 'le_1996',
 'le_1997',
 'le_1998',
 'le_1999',
 'le_2000',
 'le_2001',
 'le_2002',
 'le_2003',
 'le_2004',
 'le_2005',
 'le_2006',
 'le_2007',
 'le_2008',
 'le_2009',
 'le_2010',
 'le_2011',
 'le_2012',
 'le_2013',
 'le_2014',
 'le_2015',
 'le_2016',
 'le_2017',
 'le_2018',
 'le_2019',
 'le_2020',
 'le_2021',
 'eys_1990',
 'eys_1991',
 'eys_1992',
 'eys_1993',
 'eys_1994',
 'eys_1995',
 'eys_1996',
 'eys_1997',
 'eys_1998',
 'eys_1999',
 'eys_20

In [23]:
# Selecting the columns of interest
hdi_df = hdi_df[['country', 'hdicode', 'hdi_2021', 'region']]
hdi_df.head()

,country,hdicode,hdi_2021,region
0,Afghanistan,Low,0.478,SA
1,Angola,Medium,0.586,SSA
2,Albania,High,0.796,ECA
3,Andorra,Very High,0.858,NaN
4,United Arab Emirates,Very High,0.911,AS


In [24]:
# Renaming the columns so that they are easier to understand 
hdi_df.rename(columns={'hdi_2021': 'HDI',
                       'country': 'Country',
                       'hdicode': 'HDI Group',
                       'region': 'Region'}, inplace=True)    

In [25]:
hdi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    206 non-null    object 
 1   HDI Group  191 non-null    object 
 2   HDI        202 non-null    float64
 3   Region     151 non-null    object 
dtypes: float64(1), object(3)
memory usage: 6.6+ KB


#### Lets look at the null values

In [26]:
# Looking at missing HDI values
hdi_df[hdi_df['HDI'].isnull()]

,Country,HDI Group,HDI,Region
108,Monaco,NaN,NaN,NaN
132,Nauru,NaN,NaN,EAP
142,Korea (Democratic People's Rep. of),NaN,NaN,EAP
158,Somalia,NaN,NaN,AS


In [27]:
# Lookinz at the missing HDI Group values
hdi_df[hdi_df['HDI Group'].isnull()]

,Country,HDI Group,HDI,Region
108,Monaco,NaN,NaN,NaN
132,Nauru,NaN,NaN,EAP
142,Korea (Democratic People's Rep. of),NaN,NaN,EAP
158,Somalia,NaN,NaN,AS
195,Very high human development,NaN,0.896,NaN
196,High human development,NaN,0.754,NaN
197,Medium human development,NaN,0.636,NaN
198,Low human development,NaN,0.518,NaN
199,Arab States,NaN,0.708,NaN
200,East Asia and the Pacific,NaN,0.749,NaN


As we can see we have 4 countries with missing HDI scores. These will have to be dropped as there would be no way to fill in the data for these cells (one could try to look for other datasets). We can also observe that there are summaries of the HDI scores by region. We save these in a separate dataframe so that we can have the data on the countries in one dataframe and the data about the regions in a separate one.

In [28]:
hdi_regions_df = hdi_df[hdi_df['Country'] == ['Arab States', 'East Asia and the Pacific', 'Sub-Saharan Africa', 'Latin America and the Caribbean', 'Europe and Central Asia', 'North America', 'Oceania']]    

ValueError: ('Lengths must match to compare', (206,), (7,))

In [ ]:
iq_df = pd.read_csv('data/National_IQ.csv')
iq_df.sample(3)

,Rank,Country,Measured IQ,IQ data quality,SchAch,SA direct,SA scaled,SA data quality,Final IQ,Final IQ.1
15,16.0,Australia,98.0,12.0,534.3,101.7,100.0,16.0,99.2,99.2
92,91.0,New Caledonia,85.0,2.0,NaN,NaN,NaN,NaN,85,85.0
65,66.0,Laos,89.0,2.0,NaN,NaN,NaN,NaN,89,89.0


In [ ]:
print(hdi_df.columns)


Index(['iso3', 'country', 'hdicode', 'region', 'hdi_rank_2021', 'hdi_1990',
       'hdi_1991', 'hdi_1992', 'hdi_1993', 'hdi_1994',
       ...
       'mf_2012', 'mf_2013', 'mf_2014', 'mf_2015', 'mf_2016', 'mf_2017',
       'mf_2018', 'mf_2019', 'mf_2020', 'mf_2021'],
      dtype='object', length=1008)


# 1. Cleaning the Data


# 1. Merge Data


# 2. INTRODUCTION

In the introduction, provide the description of the problem addressed (the context of your data) and the project objectives.
Very briefly describe the analysis design and how it accomplishes the stated objectives. 
State your research hypotheses in a human-understandable language.
What  can the results be used for?

# 3. DATA CLEANING AND PREPARATION

What did you need to do to clean and prepare your dataset?
Missing values, duplicates, inconsistent data types…


# 4.  DESCRIPTIVE STATISTICS

## 4.1  Univariate analysis
Histogram and metrics introduced in class. Outliers identification. Interpret and discuss your results.

## 4.2  Bivariate analysis
Scatter plots and correlation for pairs of variables of interest. Interpret and discuss your results.



# 5.  DISCUSSION AND PRELIMINARY CONCLUSIONS 

Discuss the initial insights and how they align with the objectives set in the Introduction. Briefly address any limitations or challenges encountered in the data or analysis. Reflect on the implications of these findings and how they might guide future research directions or applications
